# Access CMIP6 zarr data from AWS using the osdf protocol and compute different precipitation statistics
- This workflow is an adaptation of https://gallery.pangeo.io/repos/pangeo-gallery/cmip6/precip_frequency_change.html
- Also see, the paper https://journals.ametsoc.org/doi/full/10.1175/JCLI-D-16-0684.1
- And the article https://climatedataguide.ucar.edu/climate-data/gpcp-daily-global-precipitation-climatology-project which inspired the workflow

## Table of Contents
- [Section 1: Introduction](#Section-1:-Introduction) 
- [Section 2: Select Dask Cluster](#Section-2:-Select-Dask-Cluster) 
- [Section 3: Data Loading](#Section-3:-Data-Loading) 
- [Section 4: Data Analysis](#Section-4:-Data-Analysis) 

## Section 1: Introduction
- Load python packkages
- Load catalog url

In [1]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm
import intake
import fsspec
import seaborn as sns
import re
import aiohttp
from dask_jobqueue import PBSCluster
import pandas as pd

/glade/derecho/scratch/harshah/tmp/ipykernel_61908/493514256.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import fsspec.implementations.http as fshttp
from pelicanfs.core import OSDFFileSystem,PelicanMap 

In [3]:
rda_scratch = '/gpfs/csfs1/collections/rda/scratch/harshah'
rda_url     =  'https://data.rda.ucar.edu/'
cat_url     = rda_url +  'harshah/intake_catalogs/osdf/cmip6-aws/cmip6-osdf-zarr.json'

## Section 2: Select Dask Cluster

#### Select the Dask cluster type
The default will be LocalCluster as that can run on any system.

If running on a HPC computer with a PBS Scheduler, set to True. Otherwise, set to False.

In [4]:
USE_PBS_SCHEDULER = True

If running on Jupyter server with Dask Gateway configured, set to True. Otherwise, set to False.

In [5]:
USE_DASK_GATEWAY = False

#### Python function for a PBS cluster

In [6]:
# Create a PBS cluster object
def get_pbs_cluster():
    """ Create cluster through dask_jobqueue.   
    """
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster(
        job_name = 'dask-osdf-24',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = rda_scratch + '/dask/spill',
        log_directory = rda_scratch + '/dask/logs/',
        resource_spec = 'select=1:ncpus=1:mem=4GB',
        queue = 'casper',
        walltime = '3:00:00',
        #interface = 'ib0'
        interface = 'ext'
    )
    return cluster

#### Python function for a Gateway Cluster

In [7]:
def get_gateway_cluster():
    """ Create cluster through dask_gateway
    """
    from dask_gateway import Gateway

    gateway = Gateway()
    cluster = gateway.new_cluster()
    cluster.adapt(minimum=2, maximum=4)
    return cluster

In [8]:
def get_local_cluster():
    """ Create cluster using the Jupyter server's resources
    """
    from distributed import LocalCluster, performance_report
    cluster = LocalCluster()    

    cluster.scale(6)
    return cluster

#### Python logic for a Local Cluster
This uses True/False boolean logic based on the variables set in the previous cells

In [9]:
# Obtain dask cluster in one of three ways
if USE_PBS_SCHEDULER:
    cluster = get_pbs_cluster()
elif USE_DASK_GATEWAY:
    cluster = get_gateway_cluster()
else:
    cluster = get_local_cluster()

# Connect to cluster
from distributed import Client
client = Client(cluster)

In [10]:
# Scale the cluster and display cluster dashboard URL
n_workers =8
cluster.scale(n_workers)
client.wait_for_workers(n_workers = n_workers)
cluster

PBSCluster(91804813, 'tcp://128.117.208.98:41245', workers=8, threads=8, memory=32.00 GiB)

## Section 3: Data Loading
- Load catalog and select data subset

In [11]:
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,709
grid_label,10
zstore,522217
dcpp_init_year,60


In [12]:
[eid for eid in col.df['experiment_id'].unique() if 'ssp' in eid]

['esm-ssp585-ssp126Lu',
 'ssp126-ssp370Lu',
 'ssp370-ssp126Lu',
 'ssp585',
 'ssp245',
 'ssp370-lowNTCF',
 'ssp370SST-ssp126Lu',
 'ssp370SST',
 'ssp370pdSST',
 'ssp370SST-lowCH4',
 'ssp370SST-lowNTCF',
 'ssp126',
 'ssp119',
 'ssp370',
 'esm-ssp585',
 'ssp245-nat',
 'ssp245-GHG',
 'ssp460',
 'ssp434',
 'ssp534-over',
 'ssp245-aer',
 'ssp245-stratO3',
 'ssp245-cov-fossil',
 'ssp245-cov-modgreen',
 'ssp245-cov-strgreen',
 'ssp245-covid',
 'ssp585-bgc']

In [13]:
# there is currently a significant amount of data for these runs
expts = ['historical', 'ssp585']

query = dict(
    experiment_id=expts,
    table_id='3hr',
    variable_id=['pr'],
    #member_id = 'r1i1p1f1',
    #activity_id = 'CMIP',
)

col_subset = col.search(require_all_on=["source_id"], **query)
col_subset

,unique
activity_id,2
institution_id,3
source_id,4
experiment_id,2
member_id,18
table_id,1
variable_id,1
grid_label,2
zstore,24
dcpp_init_year,0


In [14]:
col_subset.df.groupby("source_id")[["experiment_id", "variable_id", "table_id","activity_id"]].nunique()

,experiment_id,variable_id,table_id,activity_id
source_id,,,,
BCC-CSM2-MR,2,1,1,2
CNRM-CM6-1,2,1,1,2
CNRM-ESM2-1,2,1,1,2
IPSL-CM6A-LR,2,1,1,2


In [15]:
col_subset.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,3hr,pr,gn,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20190108
1,ScenarioMIP,BCC,BCC-CSM2-MR,ssp585,r1i1p1f1,3hr,pr,gn,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20190318
2,CMIP,CNRM-CERFACS,CNRM-CM6-1,historical,r1i1p1f2,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20180917
3,CMIP,CNRM-CERFACS,CNRM-CM6-1,historical,r2i1p1f2,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20181126
4,CMIP,CNRM-CERFACS,CNRM-CM6-1,historical,r3i1p1f2,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20190125
5,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp585,r1i1p1f2,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20190219
6,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r1i1p1f2,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20181206
7,ScenarioMIP,CNRM-CERFACS,CNRM-ESM2-1,ssp585,r1i1p1f2,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20190328
8,CMIP,IPSL,IPSL-CM6A-LR,historical,r4i1p1f1,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20180803
9,CMIP,IPSL,IPSL-CM6A-LR,historical,r18i1p1f1,3hr,pr,gr,osdf:///aws-opendata/us-west-2/cmip6-pds/CMIP6...,NaN,20180803


In [37]:
source_ids = col_subset.df['source_id'].unique()
source_ids

array(['BCC-CSM2-MR', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'IPSL-CM6A-LR'],
      dtype=object)

In [32]:
# %%time
# dsets_osdf  = col_subset.to_dataset_dict()
# print(f"\nDataset dictionary keys:\n {dsets_osdf.keys()}")

In [27]:
def load_pr_data():
    """
    Load 3hr precip data for given source and expt ids
    """
    uri = col_subset.df.zstore.values[0]

    ds = xr.open_zarr(fsspec.get_mapper(uri), consolidated=True)
    return ds

In [29]:
def load_pr_data(source_id, expt_id):
    """
    Load 3hr precip data for given source and expt ids
    """
    df_subset = col_subset.df
    uri = df_subset[(df_subset.source_id == source_id) &
                         (df_subset.experiment_id == expt_id)].zstore.values[0]

    ds = xr.open_zarr(fsspec.get_mapper(uri), consolidated=True)
    return ds

In [30]:
def precip_hist(ds, nbins=100, pr_log_min=-3, pr_log_max=2):
    """
    Calculate precipitation histogram for a single model.
    Lazy.
    """
    assert ds.pr.units == 'kg m-2 s-1'

    # mm/day
    bins_mm_day = np.hstack([[0], np.logspace(pr_log_min, pr_log_max, nbins)])
    bins_kg_m2s = bins_mm_day / (24*60*60)

    pr_hist = histogram(ds.pr, bins=[bins_kg_m2s], dim=['lon']).mean(dim='time')

    log_bin_spacing = np.diff(np.log(bins_kg_m2s[1:3])).item()
    pr_hist_norm = 100 * pr_hist / ds.dims['lon'] / log_bin_spacing
    pr_hist_norm.attrs.update({'long_name': 'zonal mean rain frequency',
                               'units': '%/Δln(r)'})
    return pr_hist_norm

def precip_hist_for_expts(dsets, experiment_ids):
    """
    Calculate histogram for a suite of experiments.
    Eager.
    """
    # actual data loading and computations happen in this next line
    pr_hists = [precip_hist(ds).load()
            for ds in [ds_hist, ds_ssp]]
    pr_hist = xr.concat(pr_hists, dim=xr.Variable('experiment_id', experiment_ids))
    return pr_hist

In [39]:
results = {}
for source_id in tqdm(source_ids):
    # get a 20 year period
    ds_hist = load_pr_data(source_id, 'historical').sel(time=slice('1980', '2000'))
    ds_ssp = load_pr_data(source_id, 'ssp585').sel(time=slice('2080', '2100'))
    pr_hist = precip_hist_for_expts([ds_hist, ds_ssp], expts)
    results[source_id] = pr_hist

  0%|          | 0/4 [00:00<?, ?it/s]

NameError: name 'histogram' is not defined

In [ ]:
#####################################################

In [17]:
def drop_all_bounds(ds):
    drop_vars = [vname for vname in ds.coords
                 if (('_bounds') in vname ) or ('_bnds') in vname]
    return ds.drop(drop_vars)

def open_dset(df):
    #assert len(df) == 1
    ds = xr.open_zarr(fsspec.get_mapper(df.zstore.values[0]), consolidated=True)
    return drop_all_bounds(ds)

def open_delayed(df):
    return dask.delayed(open_dset)(df)

from collections import defaultdict
dsets = defaultdict(dict)

for group, df in col_subset.df.groupby(by=['source_id', 'experiment_id']):
    dsets[group[0]][group[1]] = open_delayed(df)

In [18]:
dsets_ = dask.compute(dict(dsets))[0]

In [22]:
dsets_

{'BCC-CSM2-MR': {'historical': <xarray.Dataset> Size: 39GB
  Dimensions:  (lat: 160, lon: 320, time: 189800)
  Coordinates:
    * lat      (lat) float64 1kB -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
    * lon      (lon) float64 3kB 0.0 1.125 2.25 3.375 ... 355.5 356.6 357.8 358.9
    * time     (time) object 2MB 1950-01-01 01:30:00 ... 2014-12-31 22:30:00
      year     (time) int64 2MB 1950 1950 1950 1950 1950 ... 2014 2014 2014 2014
  Data variables:
      pr       (time, lat, lon) float32 39GB dask.array<chunksize=(600, 160, 320), meta=np.ndarray>
  Attributes: (12/52)
      Conventions:            CF-1.7 CMIP-6.2
      activity_id:            CMIP
      branch_method:          Standard
      branch_time_in_child:   0.0
      branch_time_in_parent:  2289.0
      cmor_version:           3.3.2
      ...                     ...
      tracking_id:            hdl:21.14100/822b921d-0abf-4bfb-b2dd-f0e7231c9185...
      variable_id:            pr
      variant_label:          r1i1p1

In [19]:
###########################

In [20]:
#calculate global means
def get_lat_name(ds):
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ds.dims) - {'time'}
    return (ds * weight).mean(other_dims)

In [21]:
expt_da = xr.DataArray(expts, dims='experiment_id', name='experiment_id',
                       coords={'experiment_id': expts})

dsets_aligned = {}

for k, v in tqdm(dsets_.items()):
    expt_dsets = v.values()
    if any([d is None for d in expt_dsets]):
        print(f"Missing experiment for {k}")
        continue

    for ds in expt_dsets:
        ds.coords['year'] = ds.time.dt.year

    # workaround for
    # https://github.com/pydata/xarray/issues/2237#issuecomment-620961663
    dsets_ann_mean = [v[expt].pipe(global_mean).swap_dims({'time': 'year'})
                             .drop_vars('time').coarsen(year=12).mean()
                      for expt in expts]

    # align everything with the 4xCO2 experiment
    dsets_aligned[k] = xr.concat(dsets_ann_mean, join='outer',dim=expt_da)

  0%|          | 0/4 [00:00<?, ?it/s]

ValueError: Could not coarsen a dimension of size 189800 with window 12 and boundary='exact'. Try a different 'boundary' option.

In [ ]:
%%time
with progress.ProgressBar():
    dsets_aligned_ = dask.compute(dsets_aligned)[0]

In [ ]:
source_ids = list(dsets_aligned_.keys())
source_da = xr.DataArray(source_ids, dims='source_id', name='source_id',
                         coords={'source_id': source_ids})

big_ds = xr.concat([ds.reset_coords(drop=True)
                    for ds in dsets_aligned_.values()],
                    dim=source_da)

big_ds

### Observational time series data for comparison with ensemble spread
<!-- obsDataURL = "https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/cru/hadcrut4/air.mon.anom.median.nc" -->

In [ ]:
#
osdf_fs = OSDFFileSystem()
print(osdf_fs)


# obs_data = '/gpfs/csfs1/collections/rda/data/harshah/osdf_data/HadCRUT.5.0.2.0.analysis.summary_series.global.monthly.nc'
# ## obs_ds = xr.open_dataset(obs_data)
#
obs_url    = '/ncar/rda/harshah/osdf_data/HadCRUT.5.0.2.0.analysis.summary_series.global.monthly.nc'
#obs_ds = xr.open_dataset(obs_data)
obs_ds = xr.open_dataset(osdf_fs.open(obs_url,mode='rb'),engine='h5netcdf').tas_mean
obs_ds

In [ ]:
# obs_url    = 'osdf:///ncar/rda/harshah/osdf_data/HadCRUT.5.0.2.0.analysis.summary_series.global.monthly.zarr'
# print(obs_url)
# #
# obs_ds = xr.open_zarr(obs_url).tas_mean
# # obs_ds

In [ ]:
# Compute annual mean temperatures anomalies
obs_gmsta = obs_ds.resample(time='YS').mean(dim='time')
obs_gmsta

## Section 4: Data Analysis
- Calculate Global Mean Surface Temperature Anomaly (GMSTA)
- Grab some observations/ ERA5 reanalysis data
- Convert xarray datasets to dataframes
- Use Seaborn to plot GMSTA

In [ ]:
df_all = big_ds.to_dataframe().reset_index()
df_all.head()

In [ ]:
# Compute anomaly w.r.t 1960-1990 baseline
# Define the baseline period
baseline_df = df_all[(df_all["year"] >= 1960) & (df_all["year"] <= 1990)]

# Compute the baseline mean
baseline_mean = baseline_df["tas"].mean()

# Compute anomalies
df_all["tas_anomaly"] = df_all["tas"] - baseline_mean
df_all

In [ ]:
obs_df = obs_gmsta.to_dataframe(name='tas_anomaly').reset_index()
# obs_df

In [ ]:
# Convert 'time' to 'year' (keeping only the year)
obs_df['year'] = obs_df['time'].dt.year

# Drop the original 'time' column since we extracted 'year'
obs_df = obs_df[['year', 'tas_anomaly']]
obs_df

In [ ]:
# Create the main plot
g = sns.relplot(data=df_all, x="year", y="tas_anomaly",
                hue='experiment_id', kind="line", errorbar="sd", aspect=2, palette="Set2")  # Adjust the color palette)

# Get the current axis from the FacetGrid
ax = g.ax

# Overlay the observational data in red
sns.lineplot(data=obs_df, x="year", y="tas_anomaly",color="red", 
             linestyle="dashed", linewidth=2,label="Observations", ax=ax)

# Adjust the legend to include observations
ax.legend(title="Experiment ID + Observations")

# Show the plot
plt.show()

In [ ]:
cluster.close()

In [ ]:
# sns.relplot(data=df_all,x="year", y="tas_anomaly", hue='experiment_id',kind="line", errorbar="sd", aspect=2)